<a href="https://colab.research.google.com/github/KhaldounN/Weather-Prediction-/blob/master/FINAL_DSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [191]:
!ip3 install flair

/bin/bash: ip3: command not found


In [192]:
!python -m spacy download en_core_web_md


     |████████████████████████████████| 96.4MB 2.9MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=60b567a42bad1190a6caa097d5c5344ce14bd6218f8634ef7457ff28c74ba8c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-3mg_z2g1/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


# Train Model to classify Emotions

In [119]:
# Import data . data from source was saparerated so I can to concatenate it 

import pandas as pd 
from sklearn.model_selection import train_test_split

df_train = pd.read_csv('/content/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test = pd.read_csv('/content/test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')
df_val = pd.read_csv('/content/val.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')
df = pd.concat([df_train,df_test, df_val])


#check data balance 
df['Sentiment'].value_counts()
df.head()

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [193]:
# We create our bag of words (bow) using our tokenizer and defining an ngram range

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english', ngram_range=(1,1), tokenizer = token.tokenize)

In [194]:
#Custom transformer using Python standard library (you could use spacy as well)
from sklearn.base import TransformerMixin 

# This function will clean the text
def clean_text(text):     
    return text.strip().lower()
    
#Custom transformer using Python standard library (you could use spacy as well)
class predictors(TransformerMixin):

    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

In [172]:
# split data for train and test 
X = df['Input']
ylabels = df['Sentiment']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,ylabels, test_size=0.30, random_state=5)

In [195]:
# intial training model 
from sklearn.naive_bayes import GaussianNB

clf= RandomForestClassifier()

In [196]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', cv),
                 ('classifier', clf)], verbose=True)

In [197]:
pipe.fit(X_train,y_train)

[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.0s
[Pipeline] ........ (step 2 of 3) Processing vectorizer, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=  18.3s


Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7f668083d8d0>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=N...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
             

In [198]:
predicted = pipe.predict(X_test)

In [199]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,classification_report

In [200]:
acc_score = metrics.accuracy_score(predicted,y_test)
prec_score = precision_score(y_test,predicted, average='macro')
recall = recall_score(y_test, predicted,average='macro')
f1 = f1_score(y_test,predicted,average='macro')
matrix = confusion_matrix(y_test,predicted)

In [201]:
print(str('Accuracy: '+'{:04.2f}'.format(acc_score*100))+'%')
print(str('Precision: '+'{:04.2f}'.format(prec_score*100))+'%')
print(str('Recall: '+'{:04.2f}'.format(recall*100))+'%')
print('F1 Score: ',f1)
print(matrix)


Accuracy: 88.47%
Precision: 85.05%
Recall: 85.21%
F1 Score:  0.850893769391874
[[ 705    9   29    5   48    3]
 [  42  576   12    2   29   40]
 [  16   21 1840   73   70   12]
 [   6    1   90  384    6    2]
 [  36   35   42    6 1625   10]
 [   0   22   18    0    7  178]]


In [202]:
test_data = ['I dont feel okay ,i am ecstatic my model works', 'I love you', 'i feel lost', 'im petrified', 'I feel frustrated']

test_result = pipe.predict(test_data)

print(test_result)

['joy' 'joy' 'sadness' 'fear' 'anger']


In [190]:
from flair.models import TextClassifier
from flair.data import Sentence
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")
import math

ModuleNotFoundError: ignored

Accuracy: 77.53%
Precision: 82.71%
Recall: 58.59%
F1 Score:  0.6191209661316305
[[ 512   14   92    2  179    0]
 [  26  430   98    2  144    1]
 [  12   12 1890   23   95    0]
 [   7    5  241  162   74    0]
 [  13   11   83    2 1644    1]
 [   4   29   92    2   84   14]]


# Defining Functions 

In [ ]:

# load book , clean data and split into sentences 
def Load_book(path):
  book = open(path, 'r').read()
  book = book.replace("\n",'')
  sentences  = book.split('.')

  return sentences 

# run sentiment analysis on each sentence in the book and save them into df_setiment 
def Analyzse_sentiment(sentences):
  tagger = TextClassifier.load('sentiment')
  df_sentiment = pd.DataFrame((np.zeros((4,int(len(sentences))))))
  for i, sentence in enumerate(sentences):
    sentence = Sentence(sentence)
    tagger.predict(sentence)
    df_sentiment[i].update(sentence.labels)


  for i , sentence in enumerate(df_sentiment.iloc[0,:]):
      try:
        df_sentiment.iloc[1,i] = sentence.value
        if sentence.value == 'NEGATIVE':
          df_sentiment.iloc[2,i] = sentence.score * (-1)
        else:
          df_sentiment.iloc[2,i] = sentence.score
      except:
        pass

  return df_sentiment


# calcualte Major and Minor sentiment and vizualize it in a dashabord style

def Display_sentiment(df_sentiment  , booktitle = ' book title'):

  l = df_sentiment.iloc[2,:]
  l = pd.DataFrame(l) 
  l['Major'] = l.iloc[:,0].rolling(40).mean()
  l['Minor'] = l.iloc[:,0].rolling(10).mean()

  fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(30, 15))
  axes[0].set_title(booktitle , fontsize = 30 )
  axes[0].plot(l['Major'])
  axes[0].set_xlabel('Sentences' , fontsize=20)
  axes[0].set_ylabel('Sentiment', fontsize=20)
  axes[0].legend(['Major Sentiment(rolling average for 30 sentences)'], fontsize=20)
  axes[0].fill_between(l.index, l['Major'] ,0 , where=l['Major'] >= 0, facecolor='lightgreen',  interpolate=True)
  axes[0].fill_between(l.index, l['Major'] ,0 , where=l['Major'] <= 0, facecolor='lightcoral',  interpolate=True)
  axes[0].set_facecolor('aliceblue')
  axes[1].plot(l['Minor'])
  axes[1].set_xlabel('Sentences' , fontsize=15)
  axes[1].set_ylabel('Sentiment' , fontsize=15)
  axes[1].fill_between(l.index, l['Minor'] ,0 , where=l['Minor'] >= 0, facecolor='lightgreen',  interpolate=True)
  axes[1].fill_between(l.index, l['Minor'] ,0 , where=l['Minor'] <= 0, facecolor='lightcoral',  interpolate=True)
  axes[1].set_facecolor('aliceblue')
  axes[1].legend(['Minor Sentiment(rolling average for 5 sentences)'] , fontsize=20)

def Analyzse_emotions(sentences):
  filename = 'Randomforest_model.sav'
  clf = pickle.load(open(filename, 'rb'))
  for i in 


  

In [ ]:
Analyzse_emotions(df)

In [ ]:

s = 'I hate you  '

filename = 'Randomforest_model.sav'
pipe = pickle.load(open(filename, 'rb'))
f = pipe.predict(s)

print(f)

In [ ]:
 sentences = Load_book('/content/Beyond good and Evil.txt')
 sentences = sentences[

In [ ]:
#df = Analyzse_sentiment(sentences)
Display_sentiment(df)